In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API key not set")

MODEL = "gpt-4.1-mini"
openai=OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message=""" 
You are a helpful assistant for an airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [5]:
def chat(message,history):
    history=[{"role":h["role"],"content":h["content"]} for h in history]
    messages = [{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response=openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [6]:
ticket_prices={"london":"$799","paris":"$899","tokyo":"$1400","berlin":"$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

In [9]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $799'

In [11]:
# there's a particular dictionary structure that's required to describe our functions

## we just describing the function "get_ticket_price" to LLM

price_function={
    "name":"get_ticket_price",
    "description":"Get the price of a return ticket to the destination city",
    "parameters":{
        "type":"object",
        "properties":{
            "destination_city":{
                "type":"string",
                "description":"The city that the customer wants to travel to",
            },
        },
        "required":["destination_city"],
        "additionalProperties": False
    }

}

In [15]:
## and this is included in a list of tools:
tools=[{"type":"function","function":price_function}]

In [16]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [34]:
def handle_tool_call(message):
    tool_call=message.tool_calls[0]
    if tool_call.function.name=="get_ticket_price":
        arguments=json.loads(tool_call.function.arguments)
        city=arguments.get('destination_city')
        price_details=get_ticket_price(city)
        response={
            "role":"tool",
            "content":price_details,
            "tool_call_id":tool_call.id
        }
    return response

In [40]:
def chat(message,history):
    history=[{"role":h["role"],"content":h["content"]} for h in history]
    messages=[{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response=openai.chat.completions.create(model=MODEL,messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        print(response.choices[0].finish_reason)
        message=response.choices[0].message
        response=handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        #for message in messages:
        #    print(message)
        response=openai.chat.completions.create(model=MODEL,messages=messages)
    
    return response.choices[0].message.content

In [41]:
gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


tool_calls
Tool called for city London


Traceback (most recent call last):
  File "c:\Users\vasip\LLMCourse\projects\llm_engineering\.venv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vasip\LLMCourse\projects\llm_engineering\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vasip\LLMCourse\projects\llm_engineering\.venv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vasip\LLMCourse\projects\llm_engineering\.venv\Lib\site-packages\gradio\blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\vasip\LLMCourse\projects\llm_engineering\.venv\Lib\site-pac

In [47]:
def handle_tool_calls(message):
    responses=[]
    for tool_call in message.tool_calls:
        if tool_call.function.name=="get_ticket_price":
            arguments=json.loads(tool_call.function.arguments)
            city =  arguments.get("destination_city")
            price_details=get_ticket_price(city)
            responses.append({
                "role":"tool",
                "content":price_details,
                "tool_call_id":tool_call.id
            })
    return responses

In [51]:
### Handling Multiple Tool Calls in 1 Response

## Handling multiple tool calls 1 after another

def chat(message,history):
    history = [{"role":h["role"],"content":h["content"]} for h in history]
    messages = [{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL,messages=messages,tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message=response.choices[0].message
        responses=handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response=openai.chat.completions.create(model=MODEL,messages=messages)
    
    return response.choices[0].message.content

In [53]:
gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


Tool called for city London


In [54]:

def chat(message,history):
    history = [{"role":h["role"],"content":h["content"]} for h in history]
    messages = [{"role":"system","content":system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL,messages=messages,tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message=response.choices[0].message
        responses=handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response=openai.chat.completions.create(model=MODEL,messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Tool called for city London
Tool called for city Paris


In [56]:
import sqlite3

In [57]:
DB="price.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('create table if not exists prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [62]:

def get_ticket_price(city):
    print(f" DATBASE TOOL CREATED: getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        print(city)
        cursor.execute('SELECT price FROM prices WHERE city = ?',(city.lower(),))
        result=cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [63]:
get_ticket_price("London")

 DATBASE TOOL CREATED: getting price for London
London


'No price data available for this city'

In [64]:
def set_ticket_price(city,price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city,price) VALUES (?,?) on conflict(city) DO UPDATE Set Price = ?',(city.lower(),price,price))
        conn.commit()

In [65]:
ticket_prices={"london":799,"paris":899, "tokyo":1420,"sydney":2999}
for city,price in ticket_prices.items():
    set_ticket_price(city,price)

In [66]:
get_ticket_price("Tokyo")

 DATBASE TOOL CREATED: getting price for Tokyo
Tokyo


'Ticket price to Tokyo is $1420.0'

In [ ]:
gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


 DATBASE TOOL CREATED: getting price for London
London
